In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (16,10)

In [2]:
def read_csv(TRAIN_PATH):
    chunksize = 5_000_000 # batch size
    
    traintypes = {
        'fare': 'float32',
        'from': 'str',
        'to': 'str',
        'p_count': 'uint8', 
        'hour': 'uint8',
        'weekday': 'uint8',
        'week': 'uint8',
        'year': 'int16',
        'dist_e': 'float32',
        'dist_t': 'float32',
        'x0': 'float32',
        'x1': 'float32',
        'y0': 'float32',
        'y1': 'float32',
    }
    
    cols = list(traintypes.keys())
    
    # loading the dataframe into list of small dataframes
    df_list = []

    for df_chunk in pd.read_csv(TRAIN_PATH, usecols=cols, dtype=traintypes, chunksize=chunksize):
        df_list.append(df_chunk) 
        df = pd.concat(df_list)
        
    return df

In [3]:
train = read_csv('../input/train_cleaned_2.csv')

In [4]:
train.head()

,fare,from,to,hour,weekday,week,year,p_count,dist_e,dist_t,x0,x1,y0,y1
0,4.5,Queens,Queens,17,0,25,2009,1,0.638322,0.759079,-73.844315,-73.841614,40.721317,40.712276
1,16.9,Manhattan,Manhattan,16,1,1,2010,1,5.213395,6.717119,-74.016045,-73.979271,40.711304,40.782005
2,5.7,Manhattan,Manhattan,0,3,33,2011,2,0.852610,1.164240,-73.982735,-73.991241,40.761269,40.750561
3,7.7,Manhattan,Manhattan,4,5,16,2012,1,1.735600,1.943343,-73.987129,-73.991570,40.733143,40.758091
4,5.3,Manhattan,Manhattan,7,1,10,2010,1,1.228196,1.658726,-73.968094,-73.956657,40.768009,40.783764


In [5]:
y = train['fare']
X = train.drop(['fare'], axis=1)
del train

from sklearn.model_selection import train_test_split
X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=0.25, random_state=379582)

del X,y

In [6]:
import tensorflow as tf

In [7]:
locations = [
    'Manhattan', 'JFK', 'LaGardia', 'Newark', 'Teterboro', 'Brooklin', 'Bronx', 'Staten', 'Queens', 'Westchester',
    'Nassau', 'Jersey', 'Bayonne', 'WStaton', 'West', 'WNewYork', 'North'        
]

# num_buckets = [0, num_buckets)

p_count = tf.feature_column.numeric_column('p_count')
taxicab = tf.feature_column.numeric_column('dist_t')
euclidean = tf.feature_column.numeric_column('dist_e')

# definition of the categorical columns
hour = tf.feature_column.categorical_column_with_identity('hour', num_buckets = 24)
weekday = tf.feature_column.categorical_column_with_identity('weekday', num_buckets = 7)
week = tf.feature_column.categorical_column_with_identity('week', num_buckets = 54)
year = tf.feature_column.categorical_column_with_vocabulary_list('year', [2009, 2010, 2011, 2012, 2013, 2014, 2015])

loc_from = tf.feature_column.categorical_column_with_vocabulary_list('from', locations)
loc_to = tf.feature_column.categorical_column_with_vocabulary_list('to', locations)

# definition of the feature crosses
from_to = tf.feature_column.crossed_column([loc_from, loc_to], len(locations)**2)
from_to_year = tf.feature_column.crossed_column([from_to, year], len(locations)**2 * 7)
from_to_year_week = tf.feature_column.crossed_column([from_to_year, week], len(locations)**2 * 7 * 54)

weekday_year = tf.feature_column.crossed_column([weekday, year], 7 * 7)
hour_weekday_year = tf.feature_column.crossed_column([hour, weekday, year], 24 * 7 * 7)

In [8]:
deep_columns = [
    # Embedding_column to "group" together ...
    tf.feature_column.embedding_column(from_to, 50),
    tf.feature_column.embedding_column(from_to_year, 50),
    tf.feature_column.embedding_column(from_to_year_week, 50),
    tf.feature_column.embedding_column(weekday_year, 50),
    tf.feature_column.embedding_column(hour_weekday_year, 50),

    # indicator columns
    tf.feature_column.indicator_column(loc_from),
    tf.feature_column.indicator_column(loc_to),
    tf.feature_column.indicator_column(hour),
    tf.feature_column.indicator_column(weekday),
    tf.feature_column.indicator_column(week),
    tf.feature_column.indicator_column(year),

    # numeric columns
    taxicab, euclidean, p_count
]

In [9]:
def train_input_fn(features, labels, batch_size):
    return tf.estimator.inputs.pandas_input_fn(
        x = features,
        y = labels,
        num_epochs = 1,
        shuffle = True,
        batch_size = batch_size)

def eval_input_fn(features, labels):
    return tf.estimator.inputs.pandas_input_fn(
        x = features,
        y = labels,
        num_epochs = 1,
        shuffle = True)

def pred_input_fn(features):
    return tf.estimator.inputs.pandas_input_fn(
        x = features,
        num_epochs = 1,
        shuffle = False)

In [10]:
model_dir = './DNN_Regressor_regions'

file_writer = tf.summary.FileWriter(model_dir)

estimator = tf.estimator.DNNRegressor(
    model_dir = model_dir,
    feature_columns = deep_columns,
    optimizer = tf.train.AdamOptimizer(learning_rate=0.0001),
    hidden_units = [128,128,128,128], 
    batch_norm = True,
    dropout = 0.1,
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './DNN_Regressor_regions', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000029995F814A8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [11]:
train_spec = tf.estimator.TrainSpec(input_fn = train_input_fn(X_train, y_train, batch_size=2500))
eval_spec = tf.estimator.EvalSpec(input_fn = eval_input_fn(X_eval, y_eval))

In [12]:
tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps None or save_checkpoints_secs 600.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ./DNN_Regressor_regions\model.ckpt.
INFO:tensorflow:loss = 544813.75, step = 0
INFO:tensorflow:global_step/sec: 12.5629
INFO:tensorflow:loss = 364666.78, step = 100 (7.961 sec)
INFO:tensorflow:global_step/sec: 13.2609
INFO:tensorflow:loss = 457731.8, step = 200 (7.541 sec)
INFO:tensorflow:global_step/sec: 14.105
INFO:tensorflow:loss = 338238.94, step = 300 (7.090 sec)
INFO:tensorflow:global_step/sec: 14.2366
INFO:tensor

INFO:tensorflow:global_step/sec: 14.4088
INFO:tensorflow:loss = 39255.344, step = 7800 (6.940 sec)
INFO:tensorflow:global_step/sec: 14.5627
INFO:tensorflow:loss = 28058.89, step = 7900 (6.867 sec)
INFO:tensorflow:global_step/sec: 14.2201
INFO:tensorflow:loss = 25665.299, step = 8000 (7.032 sec)
INFO:tensorflow:global_step/sec: 14.1806
INFO:tensorflow:loss = 26298.422, step = 8100 (7.052 sec)
INFO:tensorflow:global_step/sec: 14.3158
INFO:tensorflow:loss = 44031.66, step = 8200 (6.985 sec)
INFO:tensorflow:global_step/sec: 14.1753
INFO:tensorflow:loss = 41849.133, step = 8300 (7.055 sec)
INFO:tensorflow:global_step/sec: 14.8071
INFO:tensorflow:loss = 33343.434, step = 8400 (6.754 sec)
INFO:tensorflow:Saving checkpoints for 8497 into ./DNN_Regressor_regions\model.ckpt.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-09-04-01:40:01
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./DNN_Regress

INFO:tensorflow:loss = 34173.19, step = 14900 (6.746 sec)
INFO:tensorflow:global_step/sec: 14.7966
INFO:tensorflow:loss = 32272.367, step = 15000 (6.758 sec)
INFO:tensorflow:global_step/sec: 14.7057
INFO:tensorflow:loss = 32319.625, step = 15100 (6.800 sec)
INFO:tensorflow:global_step/sec: 14.3559
INFO:tensorflow:loss = 37977.742, step = 15200 (6.966 sec)
INFO:tensorflow:global_step/sec: 14.5514
INFO:tensorflow:loss = 27477.883, step = 15300 (6.871 sec)
INFO:tensorflow:global_step/sec: 14.2308
INFO:tensorflow:loss = 31792.121, step = 15400 (7.028 sec)
INFO:tensorflow:global_step/sec: 13.6047
INFO:tensorflow:loss = 32475.303, step = 15500 (7.350 sec)
INFO:tensorflow:global_step/sec: 14.5349
INFO:tensorflow:loss = 33406.047, step = 15600 (6.880 sec)
INFO:tensorflow:global_step/sec: 14.0009
INFO:tensorflow:loss = 33271.535, step = 15700 (7.142 sec)
INFO:tensorflow:global_step/sec: 14.4131
INFO:tensorflow:loss = 30771.11, step = 15800 (6.938 sec)
INFO:tensorflow:global_step/sec: 14.4843
IN

({'average_loss': 12.103337,
  'label/mean': 11.287052,
  'loss': 1549.2272,
  'prediction/mean': 11.243671,
  'global_step': 16108},
 [])

In [16]:
infer = pd.read_csv('../input/test_processed.csv', index_col='key')

generator = estimator.predict(input_fn=pred_input_fn(infer))
predictions = [next(generator) for i in range(len(infer))]
values = [val['predictions'].tolist()[0] for val in predictions]

ids = infer.index
submission = pd.DataFrame()
submission['fare_amount'] = values
submission['key'] = ids
submission.set_index('key', inplace=True)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./DNN_Regressor_regions\model.ckpt-16108
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [17]:
submission.head()

,fare_amount
key,
2015-01-27 13:08:24.0000002,10.895972
2015-01-27 13:08:24.0000003,11.004692
2011-10-08 11:53:44.0000002,4.999086
2012-12-01 21:12:12.0000002,9.223372
2012-12-01 21:12:12.0000003,16.294851


In [18]:
submission.to_csv('../output/19.dnn_regressor_rmse_3.47.csv')